In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 4.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
import pandas as pd
import json
from datasets import load_dataset, DatasetDict, load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from tqdm.auto import tqdm
import numpy as np
import collections
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_metric

In [4]:
notebook_login()

## Load Dataset

## Dataset can be found under the Data folder

In [ ]:
train_data = pd.read_json('/content/train.jsonl',lines=True)
validate_data = pd.read_json('/content/validation.jsonl',lines=True)

In [ ]:
#filter only phrase tags for Q&A task
train_data['tags']=train_data['tags'].apply(lambda x:x[0])
train_data=train_data[train_data['tags']=='phrase'].reset_index(drop=True)

#filter only phrase tags for Q&A task
validate_data['tags']=validate_data['tags'].apply(lambda x:x[0])
validate_data=validate_data[validate_data['tags']=='phrase'].reset_index(drop=True)

In [ ]:
# data processing to get context and spoiler start position
def create_df(dataset):

    col1=[]
    col2=[]

    for idx in range(len(dataset)):
        concatlen=0
        line_occuring=dataset['spoilerPositions'][idx][0][0][0]

        for i in range(line_occuring):
            concatlen+=len(dataset['targetParagraphs'][idx][i])


        starting_position=concatlen+dataset['spoilerPositions'][idx][0][0][1]

        col1.append({'answer_start': [starting_position], 'text': dataset['spoiler'][idx]})


        l=''
        for line in dataset['targetParagraphs'][idx]:
            l+=line
        
        col2.append(l)

    df = pd.DataFrame(list(zip(col1, col2)),
    columns =['answers', 'context'])
    df['question']=dataset['postText'].apply(lambda x: x[0])
    df['id']=dataset['postId']

    json_data=[]
    for index, row in df.iterrows():
        json_data.append({
            'question': row['question'],
            'context': row['context'],
            'answers': row['answers'],
            'id': row['id']
        })

    return json_data

In [ ]:

train_json=create_df(train_data)
validate_json=create_df(validate_data)

with open('train_json.json', 'w') as f:
    json.dump(train_json, f)

with open('validate_json.json', 'w') as f:
    json.dump(validate_json, f)

In [ ]:
# Load the JSON file as a Dataset object
train_dataset = load_dataset('json', data_files='train_json.json')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4ecfe61e1443adf2/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
validate_dataset = load_dataset('json', data_files='validate_json.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e13d5eb9b0478e01/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Create a DatasetDict object containing the Dataset
train_dict = DatasetDict({'train': train_dataset['train']})
valid_dict = DatasetDict({'valid': validate_dataset['train']})

## Data Pre-Processing

In [ ]:
model = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
tokenized_train = train_dict.map(prepare_train_features, batched=True, remove_columns=train_dict["train"].column_names)
tokenized_valid = valid_dict.map(prepare_validation_features, batched=True, remove_columns=valid_dict["valid"].column_names)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

## Training

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to us

In [ ]:
batch_size=2

In [ ]:
model_name = "roberta-large"
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:


data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_valid["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/Lahen/roberta-large-finetuned-squad into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.5k/1.32G [00:00<?, ?B/s]

Download file runs/Apr25_22-08-43_74ae0b6b4fa2/events.out.tfevents.1682460535.74ae0b6b4fa2.3842.0: 100%|######…

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Download file runs/Apr25_22-08-43_74ae0b6b4fa2/1682460535.59111/events.out.tfevents.1682460535.74ae0b6b4fa2.38…

Clean file runs/Apr25_22-08-43_74ae0b6b4fa2/events.out.tfevents.1682460535.74ae0b6b4fa2.3842.0:  20%|##       …

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file runs/Apr25_22-08-43_74ae0b6b4fa2/1682460535.59111/events.out.tfevents.1682460535.74ae0b6b4fa2.3842.…

Clean file pytorch_model.bin:   0%|          | 1.00k/1.32G [00:00<?, ?B/s]

In [ ]:
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.467300,No log


TrainOutput(global_step=1573, training_loss=1.7785797507158148, metrics={'train_runtime': 1416.7096, 'train_samples_per_second': 2.22, 'train_steps_per_second': 1.11, 'total_flos': 2190587617681920.0, 'train_loss': 1.7785797507158148, 'epoch': 1.0})

## Model

In [ ]:
trainer.save_model("roberta-trained")

Upload file pytorch_model.bin:   0%|          | 1.00/1.32G [00:00<?, ?B/s]

Upload file runs/Apr25_23-30-46_77d0f055ad23/events.out.tfevents.1682465821.77d0f055ad23.2587.0:   0%|        …

To https://huggingface.co/Lahen/roberta-large-finetuned-squad
   749dccc..6e06526  main -> main

   749dccc..6e06526  main -> main



In [ ]:
# !zip -r roberta-trained.zip '/content/roberta-trained'


  adding: content/roberta-trained/ (stored 0%)
  adding: content/roberta-trained/tokenizer_config.json (deflated 50%)
  adding: content/roberta-trained/training_args.bin (deflated 48%)
  adding: content/roberta-trained/merges.txt (deflated 53%)
  adding: content/roberta-trained/config.json (deflated 49%)
  adding: content/roberta-trained/vocab.json (deflated 59%)
  adding: content/roberta-trained/special_tokens_map.json (deflated 52%)
  adding: content/roberta-trained/pytorch_model.bin (deflated 8%)
  adding: content/roberta-trained/tokenizer.json (deflated 72%)


## Prediction

In [ ]:


def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not True:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
raw_predictions = trainer.predict(tokenized_valid['valid'])

In [ ]:
final_predictions = postprocess_qa_predictions(valid_dict['valid'], tokenized_valid['valid'], raw_predictions.predictions)

Post-processing 335 example predictions split into 758 features.


  0%|          | 0/335 [00:00<?, ?it/s]

In [ ]:
final_predictions

OrderedDict([('857559227384160256', '20%'),
             ('388301310635671553', 'Sprite'),
             ('553676574290354177', 'Smoky Paprika-Baked Garbanzo Beans'),
             ('4kt69b', 'Anthony Bourdain'),
             ('4wagdb', ''),
             ('1100188040002123_1123063127714614', ''),
             ('473452702002065409', ''),
             ('170016066732231_174009032999601', ''),
             ('474005518713769985', 'South Dakota Gov. Mike Rounds'),
             ('4kq6db', 'reduced fat sour cream'),
             ('438689079539286016', 'Edward Gorey'),
             ('403757856597372928', 'Rag & Bone'),
             ('398323375371608064', 'pixie cut'),
             ('4wennf', 'southern flying squirrel'),
             ('384053590207119361', 'antique cabinet'),
             ('590544984098635776', 'August 6th'),
             ('555426985967894529', ''),
             ('452637109074272256', ''),
             ('54zbhl', 'Arizona Republic'),
             ('379437666896728064', ''),
      

## Evaluation

In [20]:
predictions = [ v for k, v in final_predictions.items()]
references = [ex["answers"]["text"] for ex in valid_dict['valid']]

In [21]:

# remove empty string elements from predictions and references
indices_to_remove = [i for i, p in enumerate(predictions) if p == ""]
for i in reversed(indices_to_remove):
    del predictions[i]
    del references[i]

print(predictions)  
print(references) 

['20%', 'Sprite', 'Smoky Paprika-Baked Garbanzo Beans', 'Anthony Bourdain', 'South Dakota Gov. Mike Rounds', 'reduced fat sour cream', 'Edward Gorey', 'Rag & Bone', 'pixie cut', 'southern flying squirrel', 'antique cabinet', 'August 6th', 'Arizona Republic', 'Santa Clara University', 'Tim Masthay', '2,032 pounds', '$25K', 'Crazy Frog', 'Amazon', 'Dieffenbachia', 'baby food', 'Michael and Kirk', '1,4-dioxane', 'Wyoming Highway Patrol', '2.1', 'giant piece of cardboard', 'Fall of the Resistance', '4.47 billion years ago', 'Switzerland', 'make-shift tap shoes', 'Rick Snyder (R-Michigan', 'two foundations', 'Boxed', '$700 million', 'Gilbert Arenas', '"Gravity"', 'Save the Children', 'never made a dollar', 'Atlanta', 'the Israelis', 'Chiwetel Ejiofor', 'Lviv', 'joins in', 'June 12', '"Do What U Want"', '$1,000', 'please freeze your sperm', 'No perm', 'pantsuit', 'VLC', 'Liechtenstein', 'sweaters', 'Julia Roberts', 'dying his hair', 'use it as underarm deodorant', 'Arthur Brooks', 'Antenna B

### BLEU Score

In [18]:

# try with different n-gram orders
bleuscore = sentence_bleu(references, predictions, weights=[.9,0,0,0])
print(f'BLEU score:',bleuscore)

BLEU score: 0.6392522269418084


## METEOR Score

In [19]:
meteor = load_metric("meteor")

score = meteor.compute(predictions=predictions, references=references)
alpha=.6
weighted_score = (1 - alpha) * score['meteor'] + alpha * len(predictions) / len(references)

weighted_score

<ipython-input-19-b14a8ca130cb>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric("meteor")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0.6540718965044414